# **Pythonic Generative AI with Snowflake Cortex**
## Author: John Heisler
In this demo, we'll focus on using Cortex LLM Fucntions via Python (https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#using-snowflake-cortex-llm-functions-with-python).

### The Value
Rather than flip flopping between python and SQL, we want to keep all of our code directly in Python. This allows engineers that perfer python to stay there when interacting with Cortex.

### The Use Case
We'll be taking the summary of a Presidential State of the Union Addresses and asking an LLM via the Cortex Complete function to generate a three point list of highlights from the summary.

In [ ]:
CREATE DATABASE IF NOT EXISTS GEN_AI;
CREATE SCHEMA IF NOT EXISTS DEMO;
CREATE TABLE IF NOT EXISTS GEN_AI.DEMO.STATE_OF_THE_UNION (
	PRESIDENT VARCHAR(16777216),
	YEAR NUMBER(38,0),
	TITLE VARCHAR(16777216),
	SUMMARY VARCHAR(16777216),
	SENTIMENT FLOAT
);

--Clear out the table if there is data in it
TRUNCATE TABLE GEN_AI.DEMO.STATE_OF_THE_UNION;

--insert the data into the table.
INSERT INTO GEN_AI.DEMO.STATE_OF_THE_UNION VALUES 
('Donald Trump', 2019, 'Donald Trump''s Third State of the Union Address','The State of the Union address delivered by the President highlights the country''s economic progress, including job creation, wage growth, and record-low unemployment rates. He also emphasized the need for bipartisanship and cooperation to address issues such as infrastructure, healthcare, and immigration. The President also acknowledged the importance of national security and the need to protect American jobs and interests. He mentioned various achievements of his administration, including the USMCA trade deal, the US recognition of Venezuela''s interim president, and the withdrawal from the Iran nuclear deal. The address concluded with a call for unity and greatness for America.', 0.4572759),
('Donald Trump', 2018, 'Donald Trump''s Second State of the Union Address','The President delivered a State of the Union address highlighting the progress made in the last year, including job creation, tax cuts, and infrastructure development. He also discussed immigration reform, defense, and foreign policy. The speech emphasized the importance of unity and putting America first. The President recognized individuals and heroes from various fields, including law enforcement, military, and civilians. He called for an end to the visa lottery and chain migration, and for a merit-based immigration system. The President also addressed the opioid crisis and the need to strengthen America''s defense and alliances abroad.', 0.43988526),
('Donald Trump', 2017, 'Donald Trump''s First State of the Union Address','The speech emphasizes the need for unity and strength in America, highlighting recent threats against the Jewish community and vandalism of Jewish cemeteries, as well as the shooting in Kansas City. The speaker calls for putting American citizens first and renewing the American spirit, creating jobs, improving infrastructure, and reforming healthcare. The speech also touches on immigration, national security, and foreign policy, including rebuilding the military and supporting NATO allies. The ultimate goal is to make America great again and create a better future for all Americans.', 0.3023266),
('Barack Obama', 2016, 'Barack Obama''s Eighth State of the Union Address','In his final State of the Union address, President Obama called for unity and cooperation among Americans to address the challenges of the future. He highlighted the need to give everyone a fair shot at opportunity and security in the new economy, make technology work for us, keep America safe and lead the world, and make politics reflect what''s best in us. He acknowledged the economic changes that have made it harder for workers to get ahead and emphasized the importance of education, affordable college, and economic security for all. He also discussed the need to address climate change and keep America safe while avoiding isolationism or nation-building. The president urged Congress to authorize military action against ISIL and pass the Trans-Pacific Partnership, and called for an end to the practice of drawing congressional districts to favor one party. He emphasized the importance of bipartisanship and citizen engagement in the political process.', 0.28493857),
('Barack Obama', 2015, 'Barack Obama''s Seventh State of the Union Address','The speech discusses the progress made in the US over the past 15 years, including economic recovery, decreasing unemployment, and ending combat missions in Afghanistan. The speaker calls for unity and commitment to helping working families, investing in education and infrastructure, and addressing climate change. The importance of American leadership and values is emphasized.', 0.19343022),
('Barack Obama', 2014, 'Barack Obama''s Sixth State of the Union Address','The speech highlights various achievements during the past four years, including increasing graduation rates, creating jobs, reducing deficits, and improving infrastructure. The President calls for bipartisan cooperation to continue progress and create opportunities for all Americans, particularly in education and job training. He also emphasizes the importance of addressing income inequality and raising the minimum wage. The speech concludes with a call for unity and citizenship, and a commitment to ending wars and focusing on diplomacy.', 0.27800456),
('Barack Obama', 2013, 'Barack Obama''s Fifth State of the Union Address','The speech emphasizes the progress made in the US after a decade of war and a long recession, but acknowledges the need to address ongoing economic challenges such as unemployment and income inequality. The President calls for a balanced approach to deficit reduction, including tax reform and entitlement reform, and urges bipartisan cooperation to pass a budget and avoid the automatic budget cuts known as the sequester. He also discusses various initiatives to boost manufacturing, energy production, infrastructure, and education, as well as immigration reform and efforts to combat climate change. The speech concludes with a call for unity and a focus on the common good as the foundation for American progress.', 0.3029282),
('Barack Obama', 2012, 'Barack Obama''s Fourth State of the Union Address','The President delivered a speech thanking the troops for their service in Iraq and Afghanistan, and expressed optimism about the future of the American economy. He called for American manufacturing to be revitalized, and proposed tax reforms to encourage businesses to create jobs in the US. He also discussed the importance of education and innovation, and the need for energy independence. The President emphasized the importance of bipartisanship and working together to address the country''s challenges. He highlighted achievements in areas such as auto manufacturing, renewable energy, and education reform. He also addressed the need to reduce the deficit and reform the tax code, and called for an end to partisan gridlock in Congress. The President expressed confidence in America''s role as a global leader and its ability to overcome challenges.', 0.51844907),
('Barack Obama', 2011, 'Barack Obama''s Third State of the Union Address','In his State of the Union address, President Obama congratulates the new Congress and acknowledges the importance of unity and cooperation despite political differences. He emphasizes the need to focus on innovation, education, and infrastructure to win the future and create jobs. The President also calls for reducing the deficit and reorganizing the government for greater efficiency. He highlights American leadership in foreign affairs and the importance of supporting troops and their families. The speech concludes with a call to continue striving for the American Dream and the belief that ordinary people can achieve great things.', 0.45344362),
('Barack Obama', 2010, 'Barack Obama''s Second State of the Union Address','The President delivered a State of the Union address, reflecting on the country''s history of overcoming challenges and urging unity in addressing current issues. He discussed the economic recovery, job creation, financial reform, education, and health care reform. He called for bipartisanship and transparency in government, and pledged to reduce the deficit and reform entitlement programs. He also addressed foreign policy, including nuclear nonproliferation and engagement with other nations. The speech emphasized the importance of American values and ideals.', 0.3691888),
('Barack Obama', 2009, 'Barack Obama''s First State of the Union Address','The speech emphasizes the economic crisis in the United States and the need for bold action to rebuild and recover. The President acknowledges the concerns of Americans and the impact of the recession on their lives. He promises to create jobs, save or create 3.5 million jobs, and invest in areas like energy, health care, and education. The President also addresses the need to restore confidence in the financial system and re-start lending. He plans to create a new lending fund, launch a housing plan, and ensure that major banks have enough confidence and money to lend. The President calls for comprehensive health care reform and expanding education opportunities for children. He also intends to reduce the deficit and eliminate wasteful programs. The speech concludes with a call for unity and a reminder of the resilience and determination of the American people.', 0.3238493),
('George W. Bush', 2008, 'George Walker Bush''s Eighth State of the Union Address','The President delivered a State of the Union address, reflecting on the past seven years and the challenges facing the nation. He emphasized the importance of trusting individuals and empowering them to improve their lives. The speech covered various issues including the economy, taxes, spending, education, health care, energy, and foreign policy. The President called for bipartisan cooperation and urged Congress to pass legislation on several key issues, including tax relief, trade agreements, and entitlement reform. He also highlighted the progress made in Iraq and Afghanistan and the need to continue the fight against terrorism. The address concluded with a call to trust the American people and their ability to shape the course of history.', 0.22619158),
('George W. Bush', 2007, 'George Walker Bush''s Seventh State of the Union Address','The speech begins with the President expressing his honor to deliver the State of the Union address with Nancy Pelosi as Speaker of the House. He acknowledges the absence of two members of Congress and pays tribute to their recovery. He then emphasizes the need for unity and hard work to face challenges and make life better for Americans. The President highlights the economic progress, including job growth and low unemployment, and proposes three economic reforms: balancing the federal budget, reforming earmarks, and addressing entitlements. He also discusses education, health care, immigration, energy, and the justice system. The President emphasizes the importance of protecting the country from terrorism and ensuring a fair and impartial system of justice. He concludes by expressing confidence in America''s strength and resilience.', 0.6784706),
('George W. Bush', 2006, 'George Walker Bush''s Sixth State of the Union Address','The speech delivered by the President of the United States at the State of the Union address in 2006, focused on the strength of the nation, the importance of leading the world towards freedom, and the need to keep America competitive. He emphasized the need to continue the war on terror, support democratic reforms in the Middle East, and keep the economy growing. The President also highlighted the importance of education, affordable health care, and energy independence. He called for bipartisan efforts to address entitlements and immigration, and encouraged innovation and scientific research. The speech concluded with a call to finish well in the ongoing historical journey of America.', 0.3223022),
('George W. Bush', 2005, 'George Walker Bush''s Fifth State of the Union Address','The President of the United States delivered a State of the Union address, emphasizing the importance of expanding opportunities and preparing the next generation for the future. He highlighted economic achievements, including tax relief, job creation, and education reforms. The President also addressed the need for fiscal responsibility and proposed reforms to Social Security and immigration systems. He called for a stronger and more competitive economy, affordable health care, and reliable energy sources. The President also emphasized the importance of upholding traditional values and protecting the institution of marriage. He reiterated the need to keep America safe from danger and protect freedom around the world.', 0.15588917),
('George W. Bush', 2004, 'George Walker Bush''s Fourth State of the Union Address','The speech delivered by President George W. Bush to the US Congress in 2003 discussed the state of the Union, focusing on the war on terror, economic growth, and education reform. He highlighted the efforts of American servicemen and women, law enforcement personnel, and intelligence officers in protecting the country from terrorist threats. The President also praised the growth of the American economy and the passage of tax relief and compassionate reforms in education and Medicare. He emphasized the importance of continuing these efforts and remaining vigilant against terrorism, and renewing the PATRIOT Act. The speech concluded with a call for unity and faith in the power of the American people.', 0.46912265),
('George W. Bush', 2003, 'George Walker Bush''s Third State of the Union Address','The speech outlines the state of the union and the goals of the administration during this session of Congress. The goals include a growing economy, high quality and affordable health care for all Americans, promoting energy independence, and applying compassion to the deepest problems of America. The President also addresses the war on terror and the need to prevent the spread of weapons of mass destruction. He calls for a strong economy, tax relief, and energy independence, as well as compassion for the most vulnerable citizens and efforts to combat addiction and poverty. The President also emphasizes the importance of international cooperation and the need to defend freedom and human dignity around the world. He concles by addressing the threat of rogue regimes with weapons of mass destruction and the need to prevent such weapons from falling into the hands of terrorists.', 0.25700527),
('George W. Bush', 2002, 'George Walker Bush''s Second State of the Union Address','The State of the Union address delivered by President George W. Bush in 2002, following the September 11 attacks, discussed the progress made in the war on terror, the liberation of Afghanistan, and the ongoing threat of weapons of mass destruction in the hands of rogue nations. He called for unity and action against terrorism and emphasized the importance of economic and homeland security. The speech also highlighted the need for education, energy, and trade to create jobs and revitalize the economy. The President urged Americans to commit to serving their neighbors and nation through the new USA Freedom Corps.', 0.69316584),
('George W. Bush', 2001, 'George Walker Bush''s First State of the Union Address','The text is a speech given by a President outlining his new budget and governing approach. He expresses his appreciation for the invitation to speak and thanks Congress for their cooperation. He acknowledges the challenges facing America but believes the country has great resources. He invites Congress to work together to address issues such as education, health care, and the national debt. The President emphasizes the importance of education, particularly reading, and proposes increasing funding for it. He also discusses the need to keep promises to seniors regarding Medicare and Social Security. The budget prioritizes access to health care and supports military and veterans. The President also mentions his plans to improve the environment, help the poor and disadvantaged, and pay down the national debt. He proposes tax relief for families and simplifying the tax code. The speech ends with a call for bipartisanship and working together to make Americans proud of their government.', 0.4376691),
('Bill Clinton', 2000, 'William Jefferson Clinton''s Eighth State of the Union Address','The speaker expresses gratitude for the nation''s prosperity and social progress, highlighting economic growth, low unemployment and poverty rates, and record budget surpluses. They emphasize the importance of continuing to build a more perfect union and achieving great goals such as making America debt-free, ensuring quality healthcare for all, and making college affordable. The speech also touches on the need for education reform, rewarding work and strengthening families, and addressing crime and globalization. The speaker calls for bipartisanship and working together to meet these challenges.', 0.36162186);

--SANITY CHECK
SELECT * FROM GEN_AI.DEMO.STATE_OF_THE_UNION;

### Create Prompt with Context
Create a function that holds a prompt to instruct the model and accepts the data that we want to work with. 

In [ ]:
#replace this with your prompt
#in my case, I had a table with state of the union syummaries and I want to extract three key points from that. 
def generate_prompt(json):
    prompt = f"""
        <Role> 
        You are an expert at synthesizing summaries of Presidential Address Summaries into bulletted lists.
        </Role>
        
        <Data> 
        You are provided the summary of a presidential address and several other descriptors as a json object in <json_data> 
        </Data>

        <json_data>
        {json}
        </json_data>
        
        <Task>: Follow these instructions,
        1) Create bulleted list of the TOP THREE important points from the summary.
        </Task>
        
        <Output> 
        Produce a varchar output with the bulletted list. be as parsimonious as possible with the bullets
        </Output>
        """
    return prompt

### Cortex Complete
Pass the prompt (instructions with data) to an LLM with the complete function. 

Often times, you will see demonstrations that switch to SQL to call Complete. That can feel unnatural for engineers and organizations that handle all data engineering with Python. This approach allows us to stay right here in python.

### 🤯 🧠 CHECK IT OUT! 🧠 🤯 
* We generate the highlight list with the complete function for each row in our table 
* We call our generate_prompt() fucntion to pass the prompt and the json object that contains all of the columns in our row.


In [ ]:
import streamlit as st
from snowflake.cortex import Complete
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

#get a session
session = get_active_session()

#replace this with your table
database = 'GEN_AI'
schema = 'DEMO'
table = 'STATE_OF_THE_UNION'

#get the data into pandas
#this could be any data but I am using summaries of state of the unions-- replace this with whatever you want.
df = session.table(f"{database}.{schema}.{table}").to_pandas()

#each row's columns are collapsed into a json object. that is easier to pass to the model
#NOTICE: I am calling my generate_prompt function and passing my row as json.
#NOTICE that I am calling complete function here (https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)
    # VALUE CALLOUT: I could switch out any LLM I have access to (https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability)
df['Highlight'] = df.apply(lambda row: Complete('mistral-large2', generate_prompt(row.to_json())), axis = 1)

#!!look into call_udf amd call function for further efficiency gain via parallelisation!!
    # call_udf: https://docs.snowflake.com/en/developer-guide/snowpark/python/calling-functions#calling-user-defined-functions-udfs
    # call_function: https://docs.snowflake.com/en/developer-guide/snowpark/python/calling-functions#calling-system-defined-functions

#show us the data
st.dataframe(df['Highlight'])

### Conclusion
We are able to connect to snowflake cortex directly with python, meeting python data engineers where they're comfortable. You shoudlnt have to write SQL when you can write python ;)

# **Snowflake makes what your team is already doing more impactful.**